In [1]:
import cv2
from processing import *
from dt_apriltags import Detector
from pid import PID
import matplotlib.pyplot as plt

In [2]:
pid_x = PID(10, 0, 0, 100)
pid_y = PID(10, 0, 0, 100)

In [3]:
video = cv2.VideoCapture('at_AUV_vid (online-video-cutter.com).mkv')

fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_file = 'april_tags.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file, fourcc, 30, (width, height))

ret = True
count = 0
while ret:
    ret, frame = video.read()
    at_detector = Detector(families='tag36h11',
                       nthreads=1,
                       quad_decimate=1.0,
                       quad_sigma=0.0,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    tags = at_detector.detect(gray, estimate_tag_pose=False, camera_params=None, tag_size=None)

    color_img = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)

    # plt.imshow(color_img)
    # plt.show()


    data = process(color_img, pid_x, pid_y, at_detector)


    if data:
        powers = data[0]
        color_img = data[1]
        copy_img = color_img.copy()

        draw_center(copy_img)

        output_video.write(copy_img)
    else:
        output_video.write(frame)
    
      
video.release()
output_video.release()

KeyboardInterrupt: 